## Setup

Obtain earth-data access token from [https://urs.earthdata.nasa.gov/](https://urs.earthdata.nasa.gov/). Save it to `earth-data.tk`.

```bash
echo "Paste token below, then press Ctrl-D"
cat | install -m 600 /dev/stdin earth-data.tk
```

Run script:

```bash
env EARTHDATA_TOKEN="$(cat earth-data.tk)" ../scripts/setup-py-env.sh
```
This will:

1. Create `emit` environment
2. Register with jupyter (with extra environment variables)
3. Install packages

Data:
- `emit-stac.zip`
- Prepare with [collect-stac-md](collect-stac-md.ipynb)


In [ ]:
from odc.emit import dump_py_env

dump_py_env()

## Dask Client

Dask is optional, for small area loads there is little benefit for using it.

In [ ]:
import numpy as np
import xarray as xr
from dask import is_dask_collection
from distributed import Client
from distributed import wait as dask_wait
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

using_dask = False

if using_dask:
    if client := locals().get("client", None):
        client.close()
    client = Client(n_workers=1, threads_per_worker=None)
    display(client)

## Define work site

In [ ]:
from odc.geo import geom
from odc.stac import load as stac_load

sites = {
    "water": (147.05606946302976, -36.181539226879686),
    "park": (147.06574717098573, -36.218560534046865),
    "village": (147.0377486353588, -36.10157163927205),
}

sites_pts = {n: geom.point(*v, 4326) for n, v in sites.items()}
bbox = geom.unary_union([g for g in sites_pts.values()]).buffer(0.1).boundingbox.bbox

## Open FileDB of STACs

Normally this will come from STAC api.

In [ ]:
from odc.emit import open_zict_json
from pystac.item import Item as StacItem

stacs = open_zict_json("emit-stac.zip", "r")

stac_ids = [
    "EMIT_L2A_RFL_001_20230316T045121_2307503_004",
    "EMIT_L2A_RFL_001_20230316T045133_2307503_005",
]

stac_items = [StacItem.from_dict(stacs[_id]) for _id in stac_ids]

In [ ]:
if using_dask:
    # Dask mode
    assert client is not None
    print("Building Dask Graph")
    load_opts = dict(chunks={})
else:
    # Direct load with threads and progress
    print("Direct load")
    load_opts = {"pool": 4, "progress": tqdm}

xx = stac_load(
    stac_items,
    bbox=bbox,
    crs="utm",
    resolution=70,
    nodata=float("nan"),
    driver="odc.emit.EmitDriver",
    groupby="solar_day",
    bands=("elev", "reflectance"),
    **load_opts,
)
display(xx.odc.geobox, xx)

In [ ]:
if is_dask_collection(xx):
    print("Loading data into Dask cluster")
    xx = client.persist(xx)
    _ = dask_wait(xx)
else:
    print("Direct load")

## Visualize Bands 100-110

In [ ]:
roi = np.s_[0, :, :, 100:110]
rfl = xx.reflectance[roi]
if is_dask_collection(rfl):
    print(f"Extract subsection from Dask cluster: {rfl.shape}")
    rfl = rfl.compute()

_ = rfl.plot.imshow(col="wavelength", col_wrap=5, size=3, aspect=rfl.odc.aspect)

## Visualize Sample Locations

### Extract Reflectance for sample locations

- map from world coords to pixel
- round to int
- swap order from `X,Y` -> `Y,X`



In [ ]:
sites_pix = {
    n: tuple([int(round(v)) for v in xx.odc.geobox.project(pt).points[0]])[::-1]
    for n, pt in sites_pts.items()
}

# lookup full reflectance data for each pixel location
pxx = {n: xx.reflectance[0][pix_loc].compute().rename(n) for n, pix_loc in sites_pix.items()}

### Visualize with `xarray.plot.`

In [ ]:
fig, axs = plt.subplot_mosaic(
    [
        ["A", "B", "B"],
        ["A", "B", "B"],
    ],
    figsize=(12, 6),
)


ax = axs["A"]
xx.reflectance[0, :, :, 110].plot.imshow(
    robust=True,
    ax=ax,
    cmap="bone",
    add_colorbar=False,
)
ax.axis("equal")

for n, pt in sites_pts.items():
    qx, qy = pt.to_crs(xx.odc.crs).points[0]
    ax.scatter(qx, qy)

for px in pxx.values():
    px.plot(ax=axs["B"])

---------------------